<a href="https://colab.research.google.com/github/BeardedBaldo/Dog_Breed_Classifier/blob/main/DogBreedClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import tarfile
import os
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import copy
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
## download stanford dog dataset
url = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
r = requests.get(url, allow_redirects = True) 
open("images.tar", "wb").write(r.content)

In [ ]:
### untaring the dataset
tar = tarfile.open("images.tar")
tar.extractall("./")
tar.close()

In [ ]:
### check for gpus and assign to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
### image loader function and transform object

imageSize = (128, 128) if torch.cuda.is_available() else (128, 128)

loader = transforms.Compose([transforms.Resize(imageSize), 
                             transforms.ToTensor(),
                             transforms.Normalize((0.485, 0.456, 0.406),
                                                  (0.229, 0.224, 0.225))
                             ])

def imageLoader(imagePath):
  image = Image.open(imagePath)
  image = loader(image).unsqueeze(0)
  return image.to(device, torch.float)


In [ ]:
### test image load
samplePath = './Images/n02085620-Chihuahua/n02085620_10074.jpg'

sampleImage = imageLoader(samplePath)

print(type(sampleImage))
print(sampleImage.size())

newSize = (128, 128)
sampleTransform = transforms.Resize(newSize)
resizedImage = sampleTransform(sampleImage)
print(type(resizedImage))
print(resizedImage.size())

In [ ]:
### variable initialization
dataDir = "./Images"
testSplit = 0.2
valSplit = 0.2
epochs = 10
featureExtract = True
modelName = "vgg19"

In [ ]:
### load data and split into train and test

## delete 2933 from Shetland sheeps since its a png

dataDict = {"train": {
    "images": [],
    "labels": []
}, "validation": {
    "images": [],
    "labels": []    
}}
trainLabels = []
trainImages = []
mapping = []
testLabels = []
testImages = []

sampleList = ['./Images/n02108000-EntleBucher', './Images/n02111889-Samoyed']

for i, (root, dirs, files) in enumerate(os.walk(dataDir)):
  if root != dataDir:# and root in sampleList:    ### add root in sampleList condition only for testing
    name = root.split("/")[-1].split("-")[-1]
    mapping.append(name)
    print(root)
    print(name, " : ", i)
    nFiles = len(files)
    for j, f in enumerate(files):
      filePath = os.path.join(root, f)
      image = imageLoader(filePath)
      if j < (nFiles * (1 - testSplit)):
        trainImages.append(image)
        trainLabels.append(i-1)
      else:
        testImages.append(image)
        testLabels.append(i-1)
      del image

trainData = {}
trainData["trainImages"] = trainImages
trainData["trainLabels"] = trainLabels
nClasses = len(mapping)
    
print(trainData.keys())

In [ ]:
### train validation split 
trainImages, validationImages, trainLabels, validationLabels = train_test_split(trainImages, trainLabels, test_size = valSplit)
dataDict["train"]["images"] = trainImages
dataDict["validation"]["images"] = validationImages
dataDict["train"]["labels"] = trainLabels
dataDict["validation"]["labels"] = validationLabels

In [ ]:
### set requires_grad for model parameters

def setParameterRequiresGrad(model, featureExract):
  if featureExtract:
    for param in model.parameters():
      param.requires_grad = False

In [ ]:
### model initialization

def initializeModel(modelName, nClasses, featureExtract, 
                    use_pretrained = True):
  modelFt = None
  inputSize = 0

  if modelName == "resnet":
    modelFt = models.resnet50(pretrained = use_pretrained)
    setParameterRequiresGrad(modelFt, featureExtract)
    nFeatures = modelFt.fc.in_features
    modelFt.fc = nn.Linear(nFeatures, nClasses)
    inputSize = (224, 224)

  elif modelName == "vgg11":
    modelFt = models.vgg11_bn(pretrained=use_pretrained)
    setParameterRequiresGrad(modelFt, featureExtract)
    nFeatures = modelFt.classifier[6].in_features
    modelFt.classifier[6] = nn.Linear(nFeatures, nClasses)
    inputSize = (224, 224)

  elif modelName == "vgg19":
    modelFt = models.vgg19_bn(pretrained=use_pretrained)
    setParameterRequiresGrad(modelFt, featureExtract)
    nFeatures = modelFt.classifier[6].in_features
    modelFt.classifier[6] = nn.Linear(nFeatures, nClasses)
    inputSize = (224, 224)

  elif modelName == "inception":
    modelFt = models.inception_v3(pretrained = use_pretrained)
    setParameterRequiresGrad(modelFt, featureExtract)
    nFeatures = modelFt.AuxLogits.fc.in_features
    modelFt.AuxLogits.fc = nn.Linear(nFeatures, nClasses)
    nFeatures = modelFt.fc.in_features
    modelFt.fc = nn.Linear(nFeatures, nClasses)
    inputSize = (299, 299)

  else:
    print("Invalid model name, exiting...")
    exit()

  return modelFt, inputSize 

modelFt, inputSize = initializeModel(modelName, nClasses,
                                     featureExtract)
print(modelFt)

In [ ]:
### model GPU initialization, optimizer initialization

## send model to GPU
modelFt = modelFt.to(device)


## print parameters to learn
print("Parameters to learn")
if featureExtract:
  paramsUpdate = []
  for name, param in modelFt.named_parameters():
    if param.requires_grad == True:
      paramsUpdate.append(param)
      print("\t", name)
else:
  paramsUpdate = modelFt.parameters()
  for name, param in modelFt.named_parameters():
    if param.requires_grad == True:
      print("\t", name)


## initialize optimizer
optimizerFt = optim.Adam(paramsUpdate, lr = 0.001) 

In [ ]:
### model training function

def trainModel(model, dataDict, criterion, optimizer, 
               inputSize, nEpochs = 10, isInception = False):
  start = time.time()

  trainAccHistory = []
  valAccHistory = []

  bestModelWts = copy.deepcopy(model.state_dict())
  bestAcc = 0

  modelTransform = transforms.Resize(inputSize) 

  for epoch in range(nEpochs):
    print(f'Epoch {epoch}/{nEpochs - 1}')
    print('-' * 10)

    for phase in dataDict.keys():
      if phase == "train":
        model.train()
      else:
        model.eval()

      runningLoss = 0.0
      runningCorrects = 0

      inputs = dataDict[phase]["images"]
      labels = dataDict[phase]["labels"]

      for i, input in enumerate(inputs):
        input = modelTransform(input).to(device)
        label = []
        label.append(labels[i])
        label = torch.tensor(np.array(label)).to(device)
        
        ##zero gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == "train"):

          if isInception:
            output, auxOutput = model(input)
            mainLoss = criterion(output, label)
            auxLoss = criterion(auxOutput, label)
            loss = mainLoss + 0.4 * auxLoss
          else:
            output = model(input)
            loss = criterion(output, label)

          _, prediction = torch.max(output, 1)

          if phase == "train":
            loss.backward()
            optimizer.step()

        runningLoss += loss.item() * input.size(0)
        runningCorrects += torch.sum(prediction == label)

      epochLoss = runningLoss / (len(inputs))
      epochAcc = runningCorrects.double() / (len(inputs))
      
      print("{} loss: {:4f}, {} accuracy: {:4f}".format(phase,
                                                        epochLoss,
                                                        phase,
                                                        epochAcc))
      if phase == "train":
        trainAccHistory.append(epochAcc)
      else:
        valAccHistory.append(epochAcc)

      if phase == "validation" and epochAcc > bestAcc:
        bestAcc = epochAcc
        bestModelWts = copy.deepcopy(model.state_dict())
      

      

  timeTaken = time.time() - start
  print("training complete in {:.0f}m {:.0f}s".format(timeTaken // 60,
                                         timeTaken % 60))
  
  print("Best validation accuracy: {:4f}".format(bestAcc))

  model.load_state_dict(bestModelWts)

  return model, trainAccHistory, valAccHistory




In [ ]:
criterion = nn.CrossEntropyLoss()

modelFt, trainhist, valHist = trainModel(modelFt, dataDict, criterion,
                           optimizerFt, inputSize,
                           nEpochs = epochs,
                           isInception = (modelName == "inception"))

In [ ]:
###
tHist = [h.cpu().numpy() for h in trainhist]
vHist = [h.cpu().numpy() for h in valHist]

plt.xlabel("training epochs")
plt.ylabel("training accuracy")
plt.plot(range(1, epochs+1), tHist)
plt.plot(range(1, epochs+1), vHist)
plt.ylim((0, 1.))
plt.legend()
plt.show()

In [ ]:
### model evaluation

total = 0
correct = 0
modelTransform = transforms.Resize(inputSize)

with torch.no_grad():
  modelFt.eval()
  inputs = testImages
  labels = testLabels
  for i, input in enumerate(inputs):
      input = modelTransform(input).to(device)
      label = []
      label.append(labels[i])
      label = torch.tensor(np.array(label)).to(device)
      output = modelFt(input)
      _, predicted = torch.max(output, 1)
      #print("Label:", int(label[0]), ", Predicted:", int(predicted[0]))
      total += label.size(0)
      correct += (predicted==label).sum().item()

print("Testing accuracy: ", (correct/total) * 100)


In [ ]:
### prediction 
predictPath = "./Images/n02086240-Shih-Tzu/n02086240_1011.jpg"

def predictBreed(predictPath, model, mapping):
  input = imageLoader(predictPath)
  with torch.no_grad():
    input = modelTransform(input).to(device)
    output = model(input)
    _, predicted = torch.max(output, 1)
    predictedLabel = int(predicted[0])
    predictedBreed = mapping[predictedLabel]

  return predictedBreed

predictBreed = predictBreed(predictPath, modelFt, mapping)



In [ ]:
print(predictBreed)

Tzu
